In [1]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Activation, BatchNormalization, Dropout, SpatialDropout2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.regularizers import l1,l2
from sklearn.metrics import accuracy_score
import glob
import random
import shutil
import itertools

In [2]:
print(len(tf.config.experimental.list_physical_devices('GPU')))

0


In [3]:
os.getcwd()

'C:\\Users\\range\\OneDrive\\Desktop\\ML\\Projects\\Dogs-vs-cats claasification using CNN'

In [4]:
# organising into train validation and test sets
os.chdir('data/dogs-and-cats')
if os.path.isdir('Train/dogs') is False:
    os.makedirs('Train/dogs')
    os.makedirs('Train/cats')
    os.makedirs('Val/dogs')
    os.makedirs('Val/cats')
    os.makedirs('Test/dogs')
    os.makedirs('Test/cats')

    for f in random.sample(glob.glob('cat*'), 400):
        shutil.move(f, 'Train/cats')
    for f in random.sample(glob.glob('dog*'), 400):
        shutil.move(f, 'Train/dogs')
    for f in random.sample(glob.glob('cat*'), 100):
        shutil.move(f, 'Val/cats')
    for f in random.sample(glob.glob('dog*'), 100):
        shutil.move(f, 'Val/dogs')
    for f in random.sample(glob.glob('cat*'), 100):
        shutil.move(f, 'Test/cats')
    for f in random.sample(glob.glob('dog*'), 100):
        shutil.move(f, 'Test/dogs')
os.chdir('../../')

In [5]:
train_path = 'data/dogs-and-cats/Train'
val_path = 'data/dogs-and-cats/Val'
test_path = 'data/dogs-and-cats/Test'

In [6]:
train_batches = ImageDataGenerator(preprocessing_function = tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=train_path, target_size=(224,224), batch_size = 16)
val_batches = ImageDataGenerator(preprocessing_function = tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=val_path, target_size=(224,224), batch_size = 16)
test_batches = ImageDataGenerator(preprocessing_function = tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=test_path, target_size=(224,224), batch_size = 16, shuffle = False)

Found 2000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [7]:
# getting a bacth of images
imgs, labels = next(train_batches)
print(imgs.shape, labels.shape)

(16, 224, 224, 3) (16, 2)


In [8]:
# Function for Displaying images for a particular batch using opencv
# this shows the images after they have been preprocessed aaccording to the vgg16.preprocess_input function
import cv2
def display_images(imgs, labels):
    for i in range(16):
        cv2.imshow(str(labels[i]), imgs[i])
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [9]:
# Building the model
model = Sequential([Conv2D(filters = 10, kernel_size = (3, 3), activation = 'relu', padding = 'same', input_shape = (224,224,3), kernel_regularizer = l2(0.004)),
                   Conv2D(filters = 10, kernel_size = (3, 3), activation = 'relu', padding = 'same', kernel_regularizer = l2(0.004)),
                    BatchNormalization(),
                    MaxPool2D(pool_size = (2, 2), strides = 2),
                    # SpatialDropout2D(rate = 0.2),
                   Conv2D(filters = 16, kernel_size = (3, 3), activation = 'relu', padding = 'same', kernel_regularizer = l2(0.004)),
                   Conv2D(filters = 16, kernel_size = (3, 3), activation = 'relu', padding = 'same', kernel_regularizer = l2(0.004)),
                    BatchNormalization(),
                    MaxPool2D(pool_size = (2, 2), strides = 2),
                    # SpatialDropout2D(rate = 0.2),
                    Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same', kernel_regularizer = l2(0.004)),
                   Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same', kernel_regularizer = l2(0.004)),
                    MaxPool2D(pool_size = (2, 2), strides = 2),
                    Flatten(),
                   Dense(units = 64, activation = 'relu', kernel_regularizer = l2(0.01)),
                    BatchNormalization(),
                    Dropout(rate = 0.2),
                   Dense(units = 32, activation = 'relu'),
                    BatchNormalization(),
                    Dropout(rate = 0.2),
                   Dense(units = 2, activation = 'softmax')])

In [10]:
model2 = Sequential([
    Conv2D(filters = 64, padding = 'same', kernel_size = (3, 3), activation = 'relu', input_shape = (224, 224, 3), kernel_regularizer = l2(0.001)),
    BatchNormalization(),
    Conv2D(filters = 64, padding = 'same', kernel_size = (3, 3), activation = 'relu', kernel_regularizer = l2(0.001)),
    BatchNormalization(),
    MaxPool2D(pool_size = (2, 2), strides = 2),
    Conv2D(filters = 128, padding = 'same', kernel_size = (3, 3), activation = 'relu', kernel_regularizer = l2(0.001)),
    BatchNormalization(),
    Conv2D(filters = 128, padding = 'same', kernel_size = (3, 3), activation = 'relu', kernel_regularizer = l2(0.001)),
    BatchNormalization(),
    MaxPool2D(pool_size = (2, 2), strides = 2),
    Conv2D(filters = 256, padding = 'same', kernel_size = (3, 3), activation = 'relu', kernel_regularizer = l2(0.001)),
    BatchNormalization(),
    Conv2D(filters = 256, padding = 'same', kernel_size = (3, 3), activation = 'relu', kernel_regularizer = l2(0.001)),
    BatchNormalization(),
    Conv2D(filters = 256, padding = 'same', kernel_size = (3, 3), activation = 'relu', kernel_regularizer = l2(0.001)),
    BatchNormalization(),
    MaxPool2D(pool_size = (2, 2), strides = 2),
    Conv2D(filters = 512, padding = 'same', kernel_size = (3, 3), activation = 'relu', kernel_regularizer = l2(0.001)),
    BatchNormalization(),
    Conv2D(filters = 512, padding = 'same', kernel_size = (3, 3), activation = 'relu', kernel_regularizer = l2(0.001)),
    BatchNormalization(),
    Conv2D(filters = 512, padding = 'same', kernel_size = (3, 3), activation = 'relu', kernel_regularizer = l2(0.001)),
    BatchNormalization(),
    MaxPool2D(pool_size = (2, 2), strides = 2),
    Conv2D(filters = 512, padding = 'same', kernel_size = (3, 3), activation = 'relu', kernel_regularizer = l2(0.001)),
    BatchNormalization(),
    Conv2D(filters = 512, padding = 'same', kernel_size = (3, 3), activation = 'relu', kernel_regularizer = l2(0.001)),
    BatchNormalization(),
    Conv2D(filters = 512, padding = 'same', kernel_size = (3, 3), activation = 'relu', kernel_regularizer = l2(0.001)),
    BatchNormalization(),
    MaxPool2D(pool_size = (2, 2), strides = 2),
    Flatten(),
    Dense(units = 4096, activation = 'relu'),
    Dropout(0.25),
    Dense(units = 4096, activation = 'relu'),
    Dropout(0.25),
    Dense(units = 2, activation = 'softmax')
])

In [11]:
model.summary()
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 10)      280       
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 10)      910       
                                                                 
 batch_normalization (BatchN  (None, 224, 224, 10)     40        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 10)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 16)      1456      
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 16)      2

In [12]:
model.compile(optimizer = Adam(learning_rate = 0.0001), loss = categorical_crossentropy, metrics = ['accuracy'])

In [13]:
model2.compile(optimizer = Adam(learning_rate = 0.0001), loss = categorical_crossentropy, metrics = ['accuracy'])

In [14]:
model.fit(x = train_batches, validation_data = val_batches, epochs = 15, verbose = 2)

Epoch 1/15
125/125 - 89s - loss: 2.6772 - accuracy: 0.5575 - val_loss: 2.3568 - val_accuracy: 0.5625 - 89s/epoch - 714ms/step
Epoch 2/15
125/125 - 93s - loss: 2.2684 - accuracy: 0.6680 - val_loss: 2.4226 - val_accuracy: 0.5350 - 93s/epoch - 748ms/step
Epoch 3/15
125/125 - 97s - loss: 2.1016 - accuracy: 0.7325 - val_loss: 2.2818 - val_accuracy: 0.5825 - 97s/epoch - 778ms/step
Epoch 4/15
125/125 - 100s - loss: 1.9161 - accuracy: 0.7990 - val_loss: 2.1131 - val_accuracy: 0.6450 - 100s/epoch - 801ms/step
Epoch 5/15
125/125 - 108s - loss: 1.7980 - accuracy: 0.8370 - val_loss: 2.4825 - val_accuracy: 0.5300 - 108s/epoch - 861ms/step
Epoch 6/15
125/125 - 97s - loss: 1.6712 - accuracy: 0.8785 - val_loss: 2.3604 - val_accuracy: 0.5900 - 97s/epoch - 775ms/step
Epoch 7/15
125/125 - 91s - loss: 1.5554 - accuracy: 0.9140 - val_loss: 2.0155 - val_accuracy: 0.6650 - 91s/epoch - 731ms/step
Epoch 8/15


KeyboardInterrupt: 

In [ ]:
model2.fit(x = train_batches, validation_data = val_batches, epochs = 22, verbose = 2)

Epoch 1/22
125/125 - 1674s - loss: 5.7947 - accuracy: 0.5710 - val_loss: 5.4251 - val_accuracy: 0.5600 - 1674s/epoch - 13s/step
Epoch 2/22


In [125]:
model.evaluate(test_batches, verbose = 2)

50/50 - 9s - 175ms/step - accuracy: 0.6963 - loss: 1.4708


[1.4707728624343872, 0.6962500214576721]

In [ ]:
model2.evaluate(test_batches, verbose = 2)